# 03. Backpropagation from Scratch

## 학습 목표
- Chain Rule 이해
- Computational Graph와 역전파
- 간단한 MLP를 numpy로 직접 구현
- Gradient 계산 원리 이해

## 관련 면접 질문
- Backpropagation이 왜 효율적인가요?
- Vanishing/Exploding Gradient 문제는 왜 발생하나요?
- Chain Rule을 설명해주세요

---
## Pre-Quiz

### Q1. Chain Rule이란?
```
당신의 답: 
```

### Q2. Forward pass와 Backward pass의 차이는?
```
당신의 답: 
```

### Q3. Backpropagation이 brute-force 미분보다 효율적인 이유는?
```
당신의 답: 
```

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

np.random.seed(42)

# matplotlib 한글 설정 (macOS)
plt.rcParams['font.family'] = 'AppleGothic'
plt.rcParams['axes.unicode_minus'] = False

---
## 1. Chain Rule 복습

### 기본 공식
$$\frac{\partial L}{\partial x} = \frac{\partial L}{\partial y} \cdot \frac{\partial y}{\partial x}$$

### 예시: $y = f(g(x))$
$$\frac{dy}{dx} = \frac{df}{dg} \cdot \frac{dg}{dx}$$

In [ ]:
# Chain Rule 예시: y = (2x + 1)^2
# g(x) = 2x + 1
# f(g) = g^2
# dy/dx = df/dg * dg/dx = 2g * 2 = 4(2x + 1)

def f(x):
    return (2*x + 1)**2

def analytical_grad(x):
    """해석적 미분: dy/dx = 4(2x+1)"""
    return 4 * (2*x + 1)

def numerical_grad(f, x, eps=1e-5):
    """수치 미분: (f(x+h) - f(x-h)) / 2h"""
    return (f(x + eps) - f(x - eps)) / (2 * eps)

# 검증
x = 2.0
print(f"x = {x}")
print(f"해석적 미분: {analytical_grad(x)}")
print(f"수치적 미분: {numerical_grad(f, x)}")

---
## 2. Computational Graph 이해

### 간단한 연산 그래프
```
x ──┐
    ├── [×] ──> q ──┐
y ──┘              ├── [+] ──> L
         z ────────┘

q = x * y
L = q + z
```

### Forward & Backward
- **Forward**: 입력 → 출력 (값 계산)
- **Backward**: 출력 → 입력 (gradient 계산)

In [ ]:
# 간단한 연산 그래프 예시
# L = x*y + z

# Forward pass
x, y, z = 2.0, 3.0, 4.0
q = x * y      # q = 6
L = q + z      # L = 10

print("=== Forward Pass ===")
print(f"x={x}, y={y}, z={z}")
print(f"q = x*y = {q}")
print(f"L = q+z = {L}")

# Backward pass (dL/d? 계산)
dL_dL = 1.0           # 항상 1로 시작
dL_dq = dL_dL * 1     # L = q + z, dL/dq = 1
dL_dz = dL_dL * 1     # L = q + z, dL/dz = 1
dL_dx = dL_dq * y     # q = x*y, dq/dx = y
dL_dy = dL_dq * x     # q = x*y, dq/dy = x

print("\n=== Backward Pass ===")
print(f"dL/dL = {dL_dL}")
print(f"dL/dq = {dL_dq}")
print(f"dL/dz = {dL_dz}")
print(f"dL/dx = {dL_dx}")
print(f"dL/dy = {dL_dy}")

---
## 3. 기본 연산의 Local Gradient

| 연산 | Forward | Local Gradient |
|------|---------|----------------|
| 덧셈: $c = a + b$ | $c = a + b$ | $\frac{\partial c}{\partial a} = 1$, $\frac{\partial c}{\partial b} = 1$ |
| 곱셈: $c = a \times b$ | $c = a \times b$ | $\frac{\partial c}{\partial a} = b$, $\frac{\partial c}{\partial b} = a$ |
| ReLU: $y = \max(0, x)$ | $y = \max(0, x)$ | $\frac{\partial y}{\partial x} = \mathbb{1}(x > 0)$ |
| Sigmoid: $\sigma(x)$ | $\frac{1}{1+e^{-x}}$ | $\sigma(x)(1-\sigma(x))$ |

In [ ]:
# =================================================
# TODO: 기본 연산 클래스들 구현
# =================================================
#
# 각 클래스는 forward()와 backward() 메서드를 가집니다.
# forward: 값을 계산하고 backward에 필요한 값을 저장
# backward: upstream gradient를 받아 local gradient와 곱해 반환
#
# =================================================

class Multiply:
    """곱셈 연산: c = a * b"""
    
    def forward(self, a, b):
        # ============================================
        # 여기에 구현하세요
        # 1. a, b를 저장 (backward에서 사용)
        # 2. a * b 반환
        # ============================================
        pass
    
    def backward(self, dout):
        # ============================================
        # 여기에 구현하세요
        # dc/da = b, dc/db = a
        # da = dout * b, db = dout * a
        # ============================================
        pass


class Add:
    """덧셈 연산: c = a + b"""
    
    def forward(self, a, b):
        # ============================================
        # 여기에 구현하세요
        # ============================================
        pass
    
    def backward(self, dout):
        # ============================================
        # 여기에 구현하세요
        # dc/da = 1, dc/db = 1
        # ============================================
        pass


class ReLU:
    """ReLU: y = max(0, x)"""
    
    def forward(self, x):
        # ============================================
        # 여기에 구현하세요
        # mask = (x > 0) 저장
        # ============================================
        pass
    
    def backward(self, dout):
        # ============================================
        # 여기에 구현하세요
        # dy/dx = 1 if x > 0 else 0
        # ============================================
        pass


class Sigmoid:
    """Sigmoid: y = 1 / (1 + exp(-x))"""
    
    def forward(self, x):
        # ============================================
        # 여기에 구현하세요
        # y를 저장 (backward에서 사용)
        # ============================================
        pass
    
    def backward(self, dout):
        # ============================================
        # 여기에 구현하세요
        # dy/dx = y * (1 - y)
        # ============================================
        pass

In [ ]:
# ============ 테스트 ============
def test_operations():
    # Multiply
    mul = Multiply()
    out = mul.forward(3.0, 4.0)
    da, db = mul.backward(1.0)
    assert out == 12.0, f"Multiply forward 오류: {out}"
    assert da == 4.0 and db == 3.0, f"Multiply backward 오류: da={da}, db={db}"
    
    # Add
    add = Add()
    out = add.forward(3.0, 4.0)
    da, db = add.backward(1.0)
    assert out == 7.0, f"Add forward 오류: {out}"
    assert da == 1.0 and db == 1.0, f"Add backward 오류: da={da}, db={db}"
    
    # ReLU
    relu = ReLU()
    out = relu.forward(np.array([-1, 0, 2]))
    dx = relu.backward(np.array([1, 1, 1]))
    assert np.allclose(out, [0, 0, 2]), f"ReLU forward 오류: {out}"
    assert np.allclose(dx, [0, 0, 1]), f"ReLU backward 오류: {dx}"
    
    # Sigmoid
    sig = Sigmoid()
    out = sig.forward(0.0)
    dx = sig.backward(1.0)
    assert np.isclose(out, 0.5), f"Sigmoid forward 오류: {out}"
    assert np.isclose(dx, 0.25), f"Sigmoid backward 오류: {dx}"
    
    print("✅ 기본 연산 테스트 통과!")

test_operations()

---
## 4. Linear Layer 구현

### Forward
$$Y = XW + b$$
- $X$: (batch, in_features)
- $W$: (in_features, out_features)
- $b$: (out_features,)
- $Y$: (batch, out_features)

### Backward
$$\frac{\partial L}{\partial X} = \frac{\partial L}{\partial Y} \cdot W^T$$
$$\frac{\partial L}{\partial W} = X^T \cdot \frac{\partial L}{\partial Y}$$
$$\frac{\partial L}{\partial b} = \sum_{batch} \frac{\partial L}{\partial Y}$$

In [ ]:
# =================================================
# TODO: Linear Layer 구현
# =================================================

class Linear:
    def __init__(self, in_features, out_features):
        # Xavier initialization
        scale = np.sqrt(2.0 / (in_features + out_features))
        self.W = np.random.randn(in_features, out_features) * scale
        self.b = np.zeros(out_features)
        
        # Gradients
        self.dW = None
        self.db = None
    
    def forward(self, x):
        """
        Args:
            x: (batch, in_features)
        Returns:
            out: (batch, out_features)
        """
        # ============================================
        # 여기에 구현하세요
        # 1. self.x에 x 저장
        # 2. Y = X @ W + b 계산
        # ============================================
        pass
    
    def backward(self, dout):
        """
        Args:
            dout: (batch, out_features) - upstream gradient
        Returns:
            dx: (batch, in_features)
        """
        # ============================================
        # 여기에 구현하세요
        # dx = dout @ W^T
        # dW = x^T @ dout
        # db = sum(dout, axis=0)
        # ============================================
        pass

In [ ]:
# ============ 테스트 ============
def test_linear():
    linear = Linear(4, 3)
    x = np.random.randn(2, 4)  # batch=2, in=4
    
    # Forward
    out = linear.forward(x)
    assert out.shape == (2, 3), f"Forward shape 오류: {out.shape}"
    
    # Backward
    dout = np.ones((2, 3))
    dx = linear.backward(dout)
    
    assert dx.shape == x.shape, f"dx shape 오류: {dx.shape}"
    assert linear.dW.shape == linear.W.shape, f"dW shape 오류: {linear.dW.shape}"
    assert linear.db.shape == linear.b.shape, f"db shape 오류: {linear.db.shape}"
    
    print("✅ Linear 테스트 통과!")

test_linear()

---
## 5. MSE Loss 구현

$$L = \frac{1}{N} \sum_{i} (y_i - \hat{y}_i)^2$$

$$\frac{\partial L}{\partial \hat{y}} = \frac{2}{N} (\hat{y} - y)$$

In [ ]:
# =================================================
# TODO: MSE Loss 구현
# =================================================

class MSELoss:
    def forward(self, pred, target):
        """
        Args:
            pred: (batch, features) - 예측값
            target: (batch, features) - 정답
        Returns:
            loss: scalar
        """
        # ============================================
        # 여기에 구현하세요
        # self.diff = pred - target 저장
        # loss = mean((pred - target)^2)
        # ============================================
        pass
    
    def backward(self):
        """
        Returns:
            dout: (batch, features)
        """
        # ============================================
        # 여기에 구현하세요
        # dL/dpred = 2/N * (pred - target)
        # ============================================
        pass

---
## 6. 전체 MLP 학습

### 네트워크 구조
```
Input(2) → Linear(2→4) → ReLU → Linear(4→1) → Output
```

### XOR 문제 학습

In [ ]:
# =================================================
# TODO: 간단한 MLP 클래스 구현
# =================================================

class SimpleMLP:
    def __init__(self):
        # ============================================
        # 여기에 구현하세요
        # self.fc1 = Linear(2, 4)
        # self.relu = ReLU()
        # self.fc2 = Linear(4, 1)
        # self.loss_fn = MSELoss()
        # ============================================
        pass
    
    def forward(self, x):
        """순전파"""
        # ============================================
        # 여기에 구현하세요
        # ============================================
        pass
    
    def backward(self):
        """역전파"""
        # ============================================
        # 여기에 구현하세요
        # 역순으로 backward 호출
        # ============================================
        pass
    
    def update(self, lr):
        """파라미터 업데이트 (SGD)"""
        # ============================================
        # 여기에 구현하세요
        # W = W - lr * dW
        # b = b - lr * db
        # ============================================
        pass

In [ ]:
# XOR 데이터셋
X_xor = np.array([[0, 0], [0, 1], [1, 0], [1, 1]], dtype=np.float32)
y_xor = np.array([[0], [1], [1], [0]], dtype=np.float32)

# 학습
model = SimpleMLP()
lr = 0.5
epochs = 1000
losses = []

for epoch in range(epochs):
    # Forward
    pred = model.forward(X_xor)
    loss = model.loss_fn.forward(pred, y_xor)
    losses.append(loss)
    
    # Backward
    model.backward()
    
    # Update
    model.update(lr)
    
    if (epoch + 1) % 200 == 0:
        print(f"Epoch {epoch+1}, Loss: {loss:.4f}")

# 최종 예측
print("\n최종 예측:")
final_pred = model.forward(X_xor)
for i in range(4):
    print(f"  {X_xor[i]} -> {final_pred[i][0]:.4f} (정답: {y_xor[i][0]})")

In [ ]:
# Loss 곡선 시각화
plt.figure(figsize=(10, 4))
plt.plot(losses)
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('XOR 학습 Loss 곡선')
plt.grid(True)
plt.show()

---
## 7. Gradient 흐름 시각화

In [ ]:
def visualize_gradient_flow():
    """각 레이어의 gradient 크기 시각화"""
    # 학습된 모델에서 한 번 forward/backward
    model.forward(X_xor)
    model.loss_fn.forward(model.forward(X_xor), y_xor)
    model.backward()
    
    # Gradient 크기
    gradients = {
        'fc1.dW': np.abs(model.fc1.dW).mean(),
        'fc1.db': np.abs(model.fc1.db).mean(),
        'fc2.dW': np.abs(model.fc2.dW).mean(),
        'fc2.db': np.abs(model.fc2.db).mean(),
    }
    
    plt.figure(figsize=(10, 5))
    plt.bar(gradients.keys(), gradients.values())
    plt.xlabel('Parameter')
    plt.ylabel('Mean |Gradient|')
    plt.title('각 파라미터의 평균 Gradient 크기')
    plt.yscale('log')
    plt.grid(True, alpha=0.3)
    plt.show()

visualize_gradient_flow()

---
## Post-Quiz

### Q1. 왜 backward 시 연산 역순으로 진행하나요?
```
당신의 답: 
```

### Q2. ReLU의 backward에서 input이 0 이하일 때 gradient가 0인 이유는?
```
당신의 답: 
```

### Q3. 깊은 네트워크에서 gradient가 작아지는 문제(Vanishing)를 해결하는 방법은?
```
당신의 답: 
```

---
## 정답

<details>
<summary>👉 클릭하여 정답 코드 보기</summary>

```python
class Multiply:
    def forward(self, a, b):
        self.a, self.b = a, b
        return a * b
    
    def backward(self, dout):
        return dout * self.b, dout * self.a


class Add:
    def forward(self, a, b):
        return a + b
    
    def backward(self, dout):
        return dout, dout


class ReLU:
    def forward(self, x):
        self.mask = (x > 0)
        return np.maximum(0, x)
    
    def backward(self, dout):
        return dout * self.mask


class Sigmoid:
    def forward(self, x):
        self.y = 1 / (1 + np.exp(-x))
        return self.y
    
    def backward(self, dout):
        return dout * self.y * (1 - self.y)


class Linear:
    def __init__(self, in_features, out_features):
        scale = np.sqrt(2.0 / (in_features + out_features))
        self.W = np.random.randn(in_features, out_features) * scale
        self.b = np.zeros(out_features)
        self.dW = None
        self.db = None
    
    def forward(self, x):
        self.x = x
        return x @ self.W + self.b
    
    def backward(self, dout):
        self.dW = self.x.T @ dout
        self.db = np.sum(dout, axis=0)
        return dout @ self.W.T


class MSELoss:
    def forward(self, pred, target):
        self.diff = pred - target
        self.n = pred.size
        return np.mean(self.diff ** 2)
    
    def backward(self):
        return 2 * self.diff / self.n


class SimpleMLP:
    def __init__(self):
        self.fc1 = Linear(2, 4)
        self.relu = ReLU()
        self.fc2 = Linear(4, 1)
        self.loss_fn = MSELoss()
    
    def forward(self, x):
        x = self.fc1.forward(x)
        x = self.relu.forward(x)
        x = self.fc2.forward(x)
        return x
    
    def backward(self):
        dout = self.loss_fn.backward()
        dout = self.fc2.backward(dout)
        dout = self.relu.backward(dout)
        dout = self.fc1.backward(dout)
    
    def update(self, lr):
        self.fc1.W -= lr * self.fc1.dW
        self.fc1.b -= lr * self.fc1.db
        self.fc2.W -= lr * self.fc2.dW
        self.fc2.b -= lr * self.fc2.db
```

</details>

<details>
<summary>👉 Quiz 정답</summary>

### Pre-Quiz

**Q1. Chain Rule이란?**
합성 함수의 미분법. y=f(g(x))일 때 dy/dx = df/dg × dg/dx. 중간 변수를 거쳐 최종 출력에 대한 각 변수의 영향을 계산.

**Q2. Forward vs Backward**
- Forward: 입력→출력 방향으로 값 계산
- Backward: 출력→입력 방향으로 gradient 계산 (chain rule 적용)

**Q3. Backprop이 효율적인 이유**
중간 계산 결과를 재사용하기 때문. Brute-force는 각 파라미터마다 전체 네트워크를 다시 계산하지만, backprop은 한 번의 forward-backward로 모든 gradient를 계산.

### Post-Quiz

**Q1. 역순 진행 이유**
Chain rule 때문. dL/dx를 구하려면 dL/dy가 먼저 필요. 출력에서 시작해야 모든 upstream gradient가 준비됨.

**Q2. ReLU gradient가 0인 이유**
ReLU(x) = max(0, x). x≤0이면 출력이 0으로 상수 → x의 변화가 출력에 영향 없음 → gradient = 0.

**Q3. Vanishing Gradient 해결법**
- ReLU (sigmoid/tanh 대신)
- Residual Connection (skip connection)
- Batch/Layer Normalization
- 적절한 초기화 (Xavier, He)
- Gradient Clipping

</details>